# Algorithms for Uncertainty Quantification

## Tutorial 4: Sobol' indices for global sensitivity analysis

In this worksheet, we focus on the variance-based global sensitivity analysis. In particular, we focus on the computation of the Sobol' indices for global sensitivity analysis.

### Variance-based global sensitivity analysis

Variance-based sensitivity analysis provides a way to numerically quantify the contribution (take individually or in combination thereof) of uncertain inputs into output uncertainty; as the name suggests, the measure for output uncertainty is the output
variance. The uncertain inputs' contribution to the total resulted variance can be measured via the so called (total) Sobol' indices for global sensitivity analysis.

In chaospy it is very easy to compute the first order, second order, or total Sobol' indices for global sensitivity analysis. In the following code snippet, we assume that *model-gpc-approx* denotes the polynomial chaos approximation of the underlying model

$($obtained $\mathrm{v}\mathrm{i}\mathrm{a}\ \mathrm{c}\mathrm{h}\mathrm{a}\mathrm{o}\mathrm{s}\mathrm{p}\mathrm{y}$'s infrastructure, $\mathrm{i}.\mathrm{e}.\ model- gpc-$ *approx* $=cp.fit-$ *quadrature* ($\cdots)$ .

**import** chaospy as cp

....


$\neq$ *create the polynomial chaos approximation of the model* 

model-gpc-approx $=$ cp.fit-quadrature $($ . . $)$

$\neq$ *assume that the underlying input distribution is distr*

$\neq$ *first order Sobol' indices for global sensitivity analysis* 

first_order_Sobol_in $=$ cp.Sens_m (model-gpc-approx, distr)

$\neq$ *second order Sobol'indices for global sensitivity analysis*

second-order-S obol-ind $=$ cp.Sens_m2 (model_gpc_approx, distr)

$\neq$ *total Sobol' indices for global sensitivity analysis*

total_Sobel_ind $=$ cp. Sens-t (model-gpc-approx, dis tr)

### Assignment 1

Consider the model problem, the linear damped oscillator

\begin{align}
\left\{\begin{array}{l}
\frac{d^{2}y}{dt^{2}}(t)+c\frac{dy}{dt}(t)+ky(t)=f\cos(\omega_{\mathrm{O}}t)\\
y(0)=y_{0}\\
\frac{dy}{dt}(0)=y_{1}.
\end{array}\right.
\end{align}

Let $t\in[0$, 20 $], \triangle t=0.01$. The output of interest is $y(10)$ . Assume that $c\sim \mathcal{U}(0.08,0.12)$ , $k\sim \mathcal{U}(0.03,0.04)$ , $f\sim \mathcal{U}(0.08,0.12)$ , $y_{0}\sim \mathcal{U}(0.45,0.55)$ , $y_{1}\sim \mathcal{U}(-0.05,0.05)$ and $\omega_{\mathrm{O}}=$ $1.0$. Write a python$+$chaospy program to propagate the uncertainty in $(c,\ k,\ f,\ y_{0},\ y_{1})$ through the model in Eq. (1) using the generalized polynomial chaos expansion. Asses the expansion coefficients using the pseudo-spectral approach. Consider both non-sparse and sparse $5\mathrm{D}$ pseudo-spectral computation of the coefficients, constructed on Gaussian nodes. To generate multi-variate quadrature nodes and weights consider $K=5$ (for the $1\mathrm{D}$ quadrature rule); to construct multi-variate orthogonal polynomials, consider $N=3$ (for the $1\mathrm{D}$ orthogonal polynomials). Compute the first order and total Sobol' indices for global sensitivity analysis in both cases. What do you observe? Repeat the above experiment assuming that $c=0.1, k\sim \mathcal{U}(0.03,0.04)$ , $f\sim \mathcal{U}(0.08,0.12)$ , $\omega_{\mathrm{O}}\sim \mathcal{U}(0.80,1.20)$ , $y_{0}\sim \mathcal{U}(0.45,0.55)$ , $y_{1}\sim \mathcal{U}(-0.05,0.05)$ . What do you observe?

In [3]:
#Assignmnet 1
import numpy as np
import chaospy as cp
from scipy.integrate import odeint

# to use the odeint function, we need to transform the second order differential equation
# into a system of two linear equations
def model(w, t, p):
	x1, x2 		= w
	c, k, f, w 	= p

	f = [x2, f*np.cos(w*t) - k*x1 - c*x2]

	return f

# discretize the oscillator using the odeint function
def discretize_oscillator_odeint(model, atol, rtol, init_cond, args, t, t_interest):
	sol = odeint(model, init_cond, t, args=(args,), atol=atol, rtol=rtol)

	return sol[t_interest, 0]

if __name__ == '__main__':
    # relative and absolute tolerances for the ode int solver
    atol = 1e-10
    rtol = 1e-10

    # the parameters are no longer deterministic
    c_left      = 0.08
    c_right     = 0.12
    k_left      = 0.03
    k_right     = 0.04
    f_left      = 0.08
    f_right     = 0.12
    y0_left     = 0.45
    y0_right    = 0.55
    y1_left     = -0.05
    y1_right    = 0.05

    # w is deterministic
    w = 1.00

    # create uniform distribution object
    distr_c     = cp.Uniform(c_left, c_right)
    distr_k     = cp.Uniform(k_left, k_right)
    distr_f     = cp.Uniform(f_left, f_right)
    distr_y0    = cp.Uniform(y0_left, y0_right)
    distr_y1    = cp.Uniform(y1_left, y1_right)

    # create the multivariate distribution
    distr_5D = cp.J(distr_c, distr_k, distr_f, distr_y0, distr_y1)

    # quad deg 1D
    quad_deg_1D = 5
    poly_deg_1D = 2

    # time domain setup
    t_max       = 20.
    dt          = 0.01
    grid_size   = int(t_max/dt) + 1
    t           = np.array([i*dt for i in range(grid_size)])
    t_interest  = len(t)/2

    # create the orthogonal polynomials
    P = cp.orth_ttr(poly_deg_1D, distr_5D)
    

    #################### full grid computations #####################
    # get the non-sparse quadrature nodes and weight
    nodes_full, weights_full = cp.generate_quadrature(quad_deg_1D, distr_5D, rule='G', sparse=False)
    # create vector to save the solution
    sol_odeint_full  = np.zeros(len(nodes_full.T))

    # perform sparse pseudo-spectral approximation
    for j, n in enumerate(nodes_full.T):
        # each n is a vector with 5 components
        # n[0] = c, n[1] = k, c[2] = f, n[4] = y0, n[5] = y1
        init_cond               = n[3], n[4]
        args                    = n[0], n[1], n[2], w
        sol_odeint_full[j]      = discretize_oscillator_odeint(model, atol, rtol, init_cond, args, t, t_interest)

    # obtain the gpc approximation
    sol_gpc_full_approx = cp.fit_quadrature(P, nodes_full, weights_full, sol_odeint_full)

    # compute first order and total Sobol' indices
    first_order_Sobol_ind_full   = cp.Sens_m(sol_gpc_full_approx, distr_5D)
    total_Sobol_ind_full         = cp.Sens_t(sol_gpc_full_approx, distr_5D)
    ##################################################################

    #################### full grid computations #####################
    # get the sparse quadrature nodes and weight
    nodes_sparse, weights_sparse = cp.generate_quadrature(quad_deg_1D, distr_5D, rule='G', sparse=True)
    # create vector to save the solution
    sol_odeint_sparse  = np.zeros(len(nodes_sparse.T))

    # perform sparse pseudo-spectral approximation
    for j, n in enumerate(nodes_sparse.T):
        # each n is a vector with 5 components
        # n[0] = c, n[1] = k, c[2] = f, n[4] = y0, n[5] = y1
        init_cond               = n[3], n[4]
        args                    = n[0], n[1], n[2], w
        sol_odeint_sparse[j]    = discretize_oscillator_odeint(model, atol, rtol, init_cond, args, t, t_interest)

    # obtain the gpc approximation
    sol_gpc_sparse_approx = cp.fit_quadrature(P, nodes_sparse, weights_sparse, sol_odeint_sparse)

    # compute first order and total Sobol' indices
    first_order_Sobol_ind_sparse   = cp.Sens_m(sol_gpc_full_approx, distr_5D)
    total_Sobol_ind_sparse         = cp.Sens_t(sol_gpc_full_approx, distr_5D)
    ##################################################################

    print ('the number of quadrature nodes, i.e. model evaluation, for the full grid is', len(nodes_full.T))
    print ("the first order Sobol' indices are", first_order_Sobol_ind_full)  
    print ("the total Sobol' indices are", total_Sobol_ind_full)

    print ('the number of quadrature nodes, i.e. model evaluation, for the sparse grid is', len(nodes_sparse.T))
    print ("the first order Sobol' indices are", first_order_Sobol_ind_sparse)  
    print ("the total Sobol' indices are", total_Sobol_ind_sparse)

C:\Users\Marzieh\Anaconda3\lib\site-packages\ipykernel\__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


the number of quadrature nodes, i.e. model evaluation, for the full grid is 7776
the first order Sobol' indices are [  1.95813270e-02   1.02573528e-01   2.98675658e-03   5.11590924e-05
   8.69105074e-01]
the total Sobol' indices are [  2.15180242e-02   1.06440620e-01   3.00452590e-03   3.87306005e-04
   8.74351679e-01]
the number of quadrature nodes, i.e. model evaluation, for the sparse grid is 2203
the first order Sobol' indices are [  1.95813270e-02   1.02573528e-01   2.98675658e-03   5.11590924e-05
   8.69105074e-01]
the total Sobol' indices are [  2.15180242e-02   1.06440620e-01   3.00452590e-03   3.87306005e-04
   8.74351679e-01]


In [4]:
#Assignmnet 2
import numpy as np
import chaospy as cp
from scipy.integrate import odeint

# to use the odeint function, we need to transform the second order differential equation
# into a system of two linear equations
def model(w, t, p):
	x1, x2 		= w
	c, k, f, w 	= p

	f = [x2, f*np.cos(w*t) - k*x1 - c*x2]

	return f

# discretize the oscillator using the odeint function
def discretize_oscillator_odeint(model, atol, rtol, init_cond, args, t, t_interest):
	sol = odeint(model, init_cond, t, args=(args,), atol=atol, rtol=rtol)

	return sol[t_interest, 0]

if __name__ == '__main__':
    # relative and absolute tolerances for the ode int solver
    atol = 1e-10
    rtol = 1e-10

    # the parameters are no longer deterministic
    k_left      = 0.03
    k_right     = 0.04
    f_left      = 0.08
    f_right     = 0.12
    w_left      = 0.80
    w_right     = 1.20
    y0_left     = 0.45
    y0_right    = 0.55
    y1_left     = -0.05
    y1_right    = 0.05

    # c is deterministic
    c = 0.100

    # create uniform distribution object
    distr_k     = cp.Uniform(k_left, k_right)
    distr_f     = cp.Uniform(f_left, f_right)
    distr_w     = cp.Uniform(w_left, w_right)
    distr_y0    = cp.Uniform(y0_left, y0_right)
    distr_y1    = cp.Uniform(y1_left, y1_right)

    # create the multivariate distribution
    distr_5D = cp.J(distr_k, distr_f, distr_w, distr_y0, distr_y1)

    # quad deg 1D
    quad_deg_1D = 5
    poly_deg_1D = 2

    # time domain setup
    t_max       = 20.
    dt          = 0.01
    grid_size   = int(t_max/dt) + 1
    t           = np.array([i*dt for i in range(grid_size)])
    t_interest  = len(t)/2

    # create the orthogonal polynomials
    P = cp.orth_ttr(poly_deg_1D, distr_5D)
    

    #################### full grid computations #####################
    # get the non-sparse quadrature nodes and weight
    nodes_full, weights_full = cp.generate_quadrature(quad_deg_1D, distr_5D, rule='G', sparse=False)
    # create vector to save the solution
    sol_odeint_full  = np.zeros(len(nodes_full.T))

    # perform sparse pseudo-spectral approximation
    for j, n in enumerate(nodes_full.T):
        # each n is a vector with 5 components
        # n[0] = k, n[1] = f, c[2] = w, n[4] = y0, n[5] = y1
        init_cond               = n[3], n[4]
        args                    = c, n[0], n[1], n[2]
        sol_odeint_full[j]      = discretize_oscillator_odeint(model, atol, rtol, init_cond, args, t, t_interest)

    # obtain the gpc approximation
    sol_gpc_full_approx = cp.fit_quadrature(P, nodes_full, weights_full, sol_odeint_full)

    # compute first order and total Sobol' indices
    first_order_Sobol_ind_full   = cp.Sens_m(sol_gpc_full_approx, distr_5D)
    total_Sobol_ind_full         = cp.Sens_t(sol_gpc_full_approx, distr_5D)
    ##################################################################

    #################### full grid computations #####################
    # get the sparse quadrature nodes and weight
    nodes_sparse, weights_sparse = cp.generate_quadrature(quad_deg_1D, distr_5D, rule='G', sparse=True)
    # create vector to save the solution
    sol_odeint_sparse  = np.zeros(len(nodes_sparse.T))

    # perform sparse pseudo-spectral approximation
    for j, n in enumerate(nodes_sparse.T):
        # each n is a vector with 5 components
        # n[0] = k, n[1] = f, c[2] = w, n[4] = y0, n[5] = y1
        init_cond               = n[3], n[4]
        args                    = c, n[0], n[1], n[2]
        sol_odeint_sparse[j]    = discretize_oscillator_odeint(model, atol, rtol, init_cond, args, t, t_interest)

    # obtain the gpc approximation
    sol_gpc_sparse_approx = cp.fit_quadrature(P, nodes_sparse, weights_sparse, sol_odeint_sparse)

    # compute first order and total Sobol' indices
    first_order_Sobol_ind_sparse   = cp.Sens_m(sol_gpc_full_approx, distr_5D)
    total_Sobol_ind_sparse         = cp.Sens_t(sol_gpc_full_approx, distr_5D)
    ##################################################################

    print ('the number of quadrature nodes, i.e. model evaluation, for the full grid is', len(nodes_full.T))
    print ("the first order Sobol' indices are", first_order_Sobol_ind_full)  
    print ("the total Sobol' indices are", total_Sobol_ind_full)

    print ('the number of quadrature nodes, i.e. model evaluation, for the sparse grid is', len(nodes_sparse.T))
    print ("the first order Sobol' indices are", first_order_Sobol_ind_sparse)  
    print ("the total Sobol' indices are", total_Sobol_ind_sparse)

C:\Users\Marzieh\Anaconda3\lib\site-packages\ipykernel\__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


the number of quadrature nodes, i.e. model evaluation, for the full grid is 7776
the first order Sobol' indices are [  8.26864121e-02   2.63237986e-04   2.21295224e-01   4.32317248e-05
   6.90543234e-01]
the total Sobol' indices are [  8.57295159e-02   2.40443714e-03   2.23469849e-01   2.56931007e-04
   6.93307926e-01]
the number of quadrature nodes, i.e. model evaluation, for the sparse grid is 2203
the first order Sobol' indices are [  8.26864121e-02   2.63237986e-04   2.21295224e-01   4.32317248e-05
   6.90543234e-01]
the total Sobol' indices are [  8.57295159e-02   2.40443714e-03   2.23469849e-01   2.56931007e-04
   6.93307926e-01]
